### Instructions and Help
Currently, I do not have a nice, neat command for the user to enter to generate the vis (something like visualizeOutput(performance_data.csv, treeformat.csv) would be nice). In order to modify the files, use CMD+F to find filename and physically change the names to your file (and path, if needed).

This package `py_d3` has a lot of quirks so lots of patience is required when executing this notebook. The Javascript console in the browser interacts with d3/Javascript in Jupyter so feel free to use `console.log` for printing to the console. **A word of caution:** having the console open will do strange things to the Jupyter notebook, such as mess with the mouse interactivity or ruin scrolling. Use the console to read print statements and debug, then close it before you make any further edits in Jupyter.

#### Running
To run the notebook with the d3 library loaded, each cell must be executed one by one. If you get the error `d3 is not loaded yet`, but you've already run the first cell `%load_ext py_d3`, run that cell again. This is a known "quirk".

#### Errors 
**1.** If you see 

`The py_d3 extension is already loaded. To reload it, use:
  %reload_ext py_d3`
  
ignore it. That's what I've done so far.

**2.** If you see

`Javascript error adding output!
Reference error: d3 not defined`

accept it. I've had this error thrown when using d3 with other programs. Things usually work just fine, after this initial complaint. 

**3.** If you have completely screwed up the browser window(e.g. things have changed sizes, fonts are different, etc.), check your CSS. For example, I had a block that modified the divs in my d3. However, the style applies to the **whole** page so the divs in the Jupyter toolbar were being modified along with my d3 divs. Make sure you use names/classes specific to your d3 script only.

**4.** You may get various errors stating that some function x is not found (in my code `d3.tree` and `d3.linkHorizontal()` were called out at various stages. This is due to a known bug in the [runtime version of d3](https://github.com/ResidentMario/py_d3/issues/4). The default version of d3 that runs if you don't specify a version (e.g. `%%d3` only) is 3.15.11. In my code, I am using some v4 features so my code has `%%d3 4.2.2` as the top line. The problem is that there is a chance that when you run the cell, the notebook will use any of the d3 versions that you have previously declared (so in my case, it switches between 4.2.2 and 3.15.11 - hence why d3.tree breaks since it is d3.layout.tree in v3). Your best bet is to use the same version of d3 across all instances.

In [1]:
%load_ext py_d3

In [2]:
%%d3 4.13.0

<g></g>
<style>
        .node rect {
          cursor: pointer;
          fill: #fff;
          fill-opacity: 0.5;
          stroke: #3182bd;
          stroke-width: 1.5px;
        }

        .node text {
          font: 12px Courier; /*10px sans-serif;*/
          pointer-events: none;
        }

        .link {
          fill: none;
          stroke: #9ecae1;
          stroke-width: 1.5px;
        }

        .svg {
            overflow: scroll;
        }


</style>

<script>function parseNewick(a){for(var e=[],r={},s=a.split(/\s*(;|\(|\)|,|:)\s*/),t=0;t<s.length;t++){var n=s[t];switch(n){case"(":var c={};r.branchset=[c],e.push(r),r=c;break;case",":var c={};e[e.length-1].branchset.push(c),r=c;break;case")":r=e.pop();break;case":":break;default:var h=s[t-1];")"==h||"("==h||","==h?r.name=n:":"==h&&(r.length=parseFloat(n))}}return r}</script>
<script>

var margin = {top: 30, right: 20, bottom: 30, left: 20},

width = 960, //960 --katy: can I make this the same width as the magic box?
    barHeight = 30, //20,
    barWidth = (width - margin.left - margin.right) * 0.8;

var i = 0,
    duration = 400,
    root;

var diagonal = d3.linkHorizontal()
    .x(function(d) { return d.y; })
    .y(function(d) { return d.x; });

var svg = d3.select("g").append("svg")
    .attr("width", width) // + margin.left + margin.right)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
    
d3.queue()
    .defer(d3.text, "kate_output_slack.txt")
    .defer(d3.csv, "breast_cancer_prim_perf_counter_data.csv")
    .await(analyze);
    
function analyze(error, treeformat, perfdata) {
    if (error) throw error;
    //Tree structure data
    root = d3.hierarchy(parseNewick(treeformat), function(d){return d.branchset;});
    root.x0 = 0;
    root.y0 = 0;
    
    //Performance data -- need to associate w/tree. For now O(n) searching for corr. node
    
    domainTimes = [];
    // hard coded for now
    prim_inst = []; //map between primitive name and the location of it in the csv
    perfdata.map(function(d) {
        var sumTimes = (+d.time) + (+d.direct_time);
        if (sumTimes > 0) {
            domainTimes.push((+d.time) + (+d.direct_time));
        }
        
        prim_inst.push(d.primitive_instance);
    });
      
    colorTimeScale = d3.scaleLog() //went with log scale
                .domain(d3.extent(domainTimes))
                .range(["#deebf7", "#9ecae1", "#3182bd"]);

    update(root, perfdata);
    
}


function update(source, perfdata) {
    
  // Compute the flattened node list.
  var nodes = root.descendants();
  totalHeight = root.height;

  var height = Math.max(500, nodes.length * barHeight + margin.top + margin.bottom);

  d3.select("svg").transition()
      .duration(duration)
      .attr("height", height);

  d3.select(self.frameElement).transition()
      .duration(duration)
      .style("height", height + "px");

  // Compute the "layout". TODO https://github.com/d3/d3-hierarchy/issues/67
  var index = -1;
  root.eachBefore(function(n) {
    n.x = ++index * barHeight; 
    n.y = n.depth * 30;//katy 20;
  });

    function color(d) {
        /*var match = prim_inst.indexOf(d.data.name);
        if (match >= 0) {
            var sumTimes = (+perfdata[match].time) + (+perfdata[match].direct_time);
            console.log(sumTimes, colorTimeScale(sumTimes));
            return colorTimeScale((+perfdata[match].time) + (+perfdata[match].direct_time));
        }*/
        return colorTimeScale(0);
    }
  // Update the nodes…
  var node = svg.selectAll(".node")
    .data(nodes, function(d) { return d.id || (d.id = ++i); });
    
  // Determine the minimum width of the bars and the svg //katy
  var maxNameLen = 0;
  var maxDepth = 0;
  svg.selectAll(".node")
          .data(nodes, function(d){ 
              if (d.data.name.length > maxNameLen){ 
                  maxNameLen = d.data.name.length;
                } 
              if (d.depth > maxDepth){ 
                  maxDepth = d.depth;
                } 
         });
  
  if (barWidth > (maxNameLen * 3)) { barWidth = maxNameLen * 7.5;}
  if (width < (barWidth + (maxDepth * 30))) { width = (barWidth + (maxDepth * 10));} 
    
  var nodeEnter = node.enter().append("g")
      .attr("class", "node")
      .attr("transform", function(d) { return "translate(" + source.y0 + "," + source.x0 + ")"; })
      .style("opacity", 0);

  // Enter any new nodes at the parent's previous position.
  nodeEnter.append("rect")
      .attr("y", -barHeight / 2)
      .attr("height", barHeight)
      .attr("width", barWidth)
      .style("fill", color)
      .on("click", click);

  nodeEnter.append("text")
      .attr("dy", 3.5)
      .attr("dx", 5.5)
      .text(function(d) { return d.data.name; });

  // Transition nodes to their new position.
  nodeEnter.transition()
      .duration(duration)
      .attr("transform", function(d) { return "translate(" + d.y + "," + d.x + ")"; })
      .style("opacity", 1);

  node.transition()
      .duration(duration)
      .attr("transform", function(d) { return "translate(" + d.y + "," + d.x + ")"; })
      .style("opacity", 1)
    .select("rect")
      .style("fill", color);

  // Transition exiting nodes to the parent's new position.
  node.exit().transition()
      .duration(duration)
      .attr("transform", function(d) { return "translate(" + source.y + "," + source.x + ")"; })
      .style("opacity", 0)
      .remove();

  // Update the links…
  var link = svg.selectAll(".link")
    .data(root.links(), function(d) { return d.target.id; });

  // Enter any new links at the parent's previous position.
  link.enter().insert("path", "g")
      .attr("class", "link")
      .attr("d", function(d) {
        var o = {x: source.x0, y: source.y0};
        return diagonal({source: o, target: o});
      })
    .transition()
      .duration(duration)
      .attr("d", diagonal);

  // Transition links to their new position.
  link.transition()
      .duration(duration)
      .attr("d", diagonal);

  // Transition exiting nodes to the parent's new position.
  link.exit().transition()
      .duration(duration)
      .attr("d", function(d) {
        var o = {x: source.x, y: source.y};
        return diagonal({source: o, target: o});
      })
      .remove();

  // Stash the old positions for transition.
  root.each(function(d) {
    d.x0 = d.x;
    d.y0 = d.y;
  });
  

}

// Toggle children on click.
function click(d) {
  if (d.children) {
    d._children = d.children;
    d.children = null;
  } else {
    d.children = d._children;
    d._children = null;
  }

  update(d);
}
</script>